[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/bigdata_analyst_cert_v2/blob/main/part2/ch7/ch7_ex_multi_class_classification.ipynb)

# Section1

In [103]:
# 1. 문제정의
# 평가: f1 macro
# target: Credit_Score
# 최종파일: result.csv(컬럼 1개 pred)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd

# train = pd.read_csv("score_train.csv")
# test = pd.read_csv("score_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/score_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/score_test.csv")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4198 entries, 0 to 4197
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       4198 non-null   float64
 1   Num_of_Delayed_Payment    4198 non-null   float64
 2   Num_Credit_Inquiries      4198 non-null   float64
 3   Credit_Utilization_Ratio  4198 non-null   float64
 4   Credit_History_Age        4198 non-null   float64
 5   Payment_of_Min_Amount     4198 non-null   object 
 6   Amount_invested_monthly   4198 non-null   float64
 7   Monthly_Balance           4198 non-null   float64
 8   Credit_Mix                4198 non-null   object 
 9   Payment_Behaviour         4198 non-null   object 
 10  Age                       4198 non-null   float64
 11  Annual_Income             4198 non-null   float64
 12  Num_Bank_Accounts         4198 non-null   float64
 13  Num_Credit_Card           4198 non-null   float64
 14  Interest

In [104]:
y_train = train.pop('Credit_Score')

In [5]:
train.describe(include='O')

,Payment_of_Min_Amount,Credit_Mix,Payment_Behaviour
count,4198,4198,4198
unique,3,3,6
top,Yes,Standard,Low_spent_Small_value_payments
freq,2234,1967,1415


In [6]:
test.describe(include='O')

,Payment_of_Min_Amount,Credit_Mix,Payment_Behaviour
count,1499,1499,1499
unique,3,3,6
top,Yes,Standard,Low_spent_Small_value_payments
freq,778,685,476


In [8]:
test.isnull().sum()

,0
Delay_from_due_date,0
Num_of_Delayed_Payment,0
Num_Credit_Inquiries,0
Credit_Utilization_Ratio,0
Credit_History_Age,0
Payment_of_Min_Amount,0
Amount_invested_monthly,0
Monthly_Balance,0
Credit_Mix,0
Payment_Behaviour,0


In [105]:
# one-hot encoding
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [87]:
# encoding
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
cols = train.select_dtypes(include='object').columns

for col in cols:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])

In [88]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4198 entries, 0 to 4197
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       4198 non-null   float64
 1   Num_of_Delayed_Payment    4198 non-null   float64
 2   Num_Credit_Inquiries      4198 non-null   float64
 3   Credit_Utilization_Ratio  4198 non-null   float64
 4   Credit_History_Age        4198 non-null   float64
 5   Payment_of_Min_Amount     4198 non-null   int64  
 6   Amount_invested_monthly   4198 non-null   float64
 7   Monthly_Balance           4198 non-null   float64
 8   Credit_Mix                4198 non-null   int64  
 9   Payment_Behaviour         4198 non-null   int64  
 10  Age                       4198 non-null   float64
 11  Annual_Income             4198 non-null   float64
 12  Num_Bank_Accounts         4198 non-null   float64
 13  Num_Credit_Card           4198 non-null   float64
 14  Interest

In [57]:
# scaleing
cols = train.select_dtypes(include=['int64', 'float64']).columns

sc = StandardScaler()
train[cols] = sc.fit_transform(train[cols])
test[cols] = sc.transform(test[cols])

In [106]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=0)

In [107]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred_r = rf.predict(X_val)

In [119]:
import xgboost
from sklearn.preprocessing import LabelEncoder

# Label encode the target variable for XGBoost
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

xg = xgboost.XGBClassifier(random_state=0)
xg.fit(X_train, y_train_encoded)
pred_x = xg.predict(X_val)
pred_x


mapping = {
    'Good': 0,
    'Standard': 1,
    'Poor': 2,
}
inv_mapping = {v: k for k, v in mapping.items()}  # {0:'Good', 1:'Standard', 2:'Poor'}

pred_label = [inv_mapping[i] for i in pred_x]   # [0,1,2] → ['Good', 'Standard', ...]
pred_label


['Poor',
 'Standard',
 'Standard',
 'Standard',
 'Poor',
 'Poor',
 'Standard',
 'Good',
 'Standard',
 'Standard',
 'Poor',
 'Poor',
 'Poor',
 'Poor',
 'Poor',
 'Good',
 'Poor',
 'Standard',
 'Good',
 'Poor',
 'Poor',
 'Poor',
 'Standard',
 'Poor',
 'Poor',
 'Poor',
 'Good',
 'Poor',
 'Good',
 'Good',
 'Standard',
 'Poor',
 'Standard',
 'Good',
 'Standard',
 'Good',
 'Poor',
 'Poor',
 'Poor',
 'Standard',
 'Poor',
 'Poor',
 'Standard',
 'Poor',
 'Poor',
 'Poor',
 'Poor',
 'Poor',
 'Poor',
 'Good',
 'Poor',
 'Poor',
 'Standard',
 'Good',
 'Poor',
 'Poor',
 'Poor',
 'Poor',
 'Standard',
 'Standard',
 'Standard',
 'Good',
 'Poor',
 'Good',
 'Poor',
 'Good',
 'Standard',
 'Poor',
 'Poor',
 'Poor',
 'Standard',
 'Standard',
 'Standard',
 'Good',
 'Poor',
 'Poor',
 'Poor',
 'Poor',
 'Standard',
 'Poor',
 'Good',
 'Poor',
 'Standard',
 'Poor',
 'Standard',
 'Good',
 'Good',
 'Poor',
 'Poor',
 'Standard',
 'Poor',
 'Poor',
 'Good',
 'Poor',
 'Good',
 'Good',
 'Standard',
 'Poor',
 'Standard',
 

In [115]:
import lightgbm
lg = lightgbm.LGBMClassifier(random_state=0)
lg.fit(X_train, y_train)
pred_l = lg.predict(X_val)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001425 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2532
[LightGBM] [Info] Number of data points in the train set: 3358, number of used features: 29
[LightGBM] [Info] Start training from score -1.744076
[LightGBM] [Info] Start training from score -1.240774
[LightGBM] [Info] Start training from score -0.623559


array(['Standard', 'Poor', 'Poor', 'Poor', 'Standard', 'Standard', 'Poor',
       'Good', 'Poor', 'Poor', 'Standard', 'Standard', 'Standard',
       'Standard', 'Poor', 'Good', 'Standard', 'Poor', 'Standard',
       'Standard', 'Standard', 'Standard', 'Poor', 'Standard', 'Standard',
       'Standard', 'Good', 'Standard', 'Good', 'Good', 'Poor', 'Standard',
       'Poor', 'Good', 'Poor', 'Good', 'Standard', 'Standard', 'Standard',
       'Poor', 'Standard', 'Standard', 'Poor', 'Standard', 'Standard',
       'Poor', 'Good', 'Standard', 'Standard', 'Good', 'Standard',
       'Standard', 'Poor', 'Good', 'Standard', 'Standard', 'Standard',
       'Standard', 'Poor', 'Poor', 'Poor', 'Good', 'Standard', 'Good',
       'Standard', 'Standard', 'Poor', 'Standard', 'Standard', 'Standard',
       'Poor', 'Poor', 'Poor', 'Good', 'Standard', 'Standard', 'Standard',
       'Standard', 'Poor', 'Standard', 'Good', 'Standard', 'Standard',
       'Poor', 'Poor', 'Good', 'Good', 'Good', 'Standard', 'Poor'

In [121]:
from sklearn.metrics import f1_score

f1_r = f1_score(y_val, pred_r, average='macro')
f1_l = f1_score(y_val, pred_l, average='macro')
f1_x = f1_score(y_val, pred_x, average='macro')
print(f1_r, f1_l, f1_x)

ValueError: Mix of label input types (string and number)

In [79]:
import pandas as pd

# 데이터 불러오기
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/score_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/score_test.csv")

# 타깃 분리
target = train.pop('Credit_Score')

# 스케일링 (수치형만)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols = train.select_dtypes(include=['int64', 'float64']).columns
train[cols] = scaler.fit_transform(train[cols])
test[cols]  = scaler.transform(test[cols])

# 원핫 인코딩
train = pd.get_dummies(train)
test  = pd.get_dummies(test)

# train/test 더미 컬럼 맞추기 (중요)
train, test = train.align(test, join='left', axis=1, fill_value=0)

# 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0, stratify=target)

# 1) 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred_rf = rf.predict(X_val)

# 2) XGBoost 분류기
import xgboost as xgb
xg = xgb.XGBClassifier(
    random_state=0,
    objective='multi:softprob'  # 다중분류용 (softprob: 확률 출력)
)
xg.fit(X_tr, y_tr)
pred_x = xg.predict(X_val)

# F1-macro 계산
from sklearn.metrics import f1_score
f1_rf = f1_score(y_val, pred_rf, average='macro')
f1_x  = f1_score(y_val, pred_x,  average='macro')
print('RF f1-macro:', f1_rf)
print('XGB f1-macro:', f1_x)

# 최종 제출 (랜덤포레스트 기준)
pred_test = rf.predict(test)
submit = pd.DataFrame({'pred': pred_test})
submit.to_csv("result.csv", index=False)


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2], got ['Good' 'Poor' 'Standard']

In [116]:
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result.csv',index=False)
pd.read_csv('result.csv')

,pred
0,Poor
1,Good
2,Standard
3,Good
4,Standard
...,...
1494,Standard
1495,Poor
1496,Standard
1497,Good


### 베이스라인

In [ ]:
# 1. 문제정의
# 평가: f1 macro
# target: Credit_Score
# 최종파일: result.csv(컬럼 1개 pred)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd

# train = pd.read_csv("score_train.csv")
# test = pd.read_csv("score_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/score_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/score_test.csv")

# 3. 탐색적 데이터 분석(EDA)
print("===== 데이터 크기 =====")
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)
print("\n") # 줄 바꿈

print("===== 데이터 정보(자료형) =====")
print(train.info())
print("\n")

print("===== train 결측치 수 =====")
print(train.isnull().sum().sum())
print("\n")

print("===== test 결측치 수 =====")
print(test.isnull().sum().sum())
print("\n")

print("===== target 빈도 =====")
print(train['Credit_Score'].value_counts())

===== 데이터 크기 =====
Train Shape: (4198, 21)
Test Shape: (1499, 20)


===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4198 entries, 0 to 4197
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       4198 non-null   float64
 1   Num_of_Delayed_Payment    4198 non-null   float64
 2   Num_Credit_Inquiries      4198 non-null   float64
 3   Credit_Utilization_Ratio  4198 non-null   float64
 4   Credit_History_Age        4198 non-null   float64
 5   Payment_of_Min_Amount     4198 non-null   object 
 6   Amount_invested_monthly   4198 non-null   float64
 7   Monthly_Balance           4198 non-null   float64
 8   Credit_Mix                4198 non-null   object 
 9   Payment_Behaviour         4198 non-null   object 
 10  Age                       4198 non-null   float64
 11  Annual_Income             4198 non-null   float64
 12  Num_Bank_Accounts         

In [ ]:
# 4. 데이터 전처리
# 원핫인코딩 (target컬럼이 object형이라 제외)
target = train.pop('Credit_Score')

train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

print("\n ===== 분할된 데이터 크기 =====")
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import f1_score
f1 = f1_score(y_val, pred, average='macro')
print('\n f1-macro:', f1)

# 7. 예측 및 결과 파일 생성
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)

# 제출파일 확인
print("\n ===== 제출파일 (샘플 5개) =====")
print(pd.read_csv("result.csv").head())


 ===== 분할된 데이터 크기 =====
(3358, 29) (840, 29) (3358,) (840,)

 f1-macro: 0.7004593488873695

 ===== 제출파일 (샘플 5개) =====
       pred
0      Poor
1      Good
2  Standard
3      Good
4  Standard


### 성능개선

In [78]:
# 2. 라이브러리 및 데이터 불러오기
import pandas as pd

# train = pd.read_csv("score_train.csv")
# test = pd.read_csv("score_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/score_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/score_test.csv")

# 4. 데이터 전처리
target = train.pop('Credit_Score')

# 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols = train.select_dtypes(include=['int', 'float']).columns
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])

# 원핫인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 레이블 인코딩
# target = train.pop('Credit_Score')
# from sklearn.preprocessing import LabelEncoder
# cols = train.select_dtypes(include='object').columns
# for col in cols:
#     le = LabelEncoder()
#     train[col] = le.fit_transform(train[col])
#     test[col] = le.transform(test[col])

# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

import xgboost
xg = xgboost.XGBClassifier(random_state=0)
xg.fit(X_tr, y_tr)
pred_x = xg.predict(X_val)

from sklearn.metrics import f1_score
f1 = f1_score(y_val, pred, average='macro')
f1_x = f1_score(y_val, pred_x, average='macro')
print('f1-macro:', f1)

# 7. 예측 및 결과 파일 생성
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)



ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2], got ['Good' 'Poor' 'Standard']

# Section2

In [163]:
# 1. 문제정의
# 평가: f1-macro
# target: Drug
# 최종파일: result.csv(컬럼 1개 pred, 1확률값)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
# train = pd.read_csv("drug_train.csv")
# test = pd.read_csv("drug_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/drug_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/drug_test.csv")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          100 non-null    int64  
 1   Sex          100 non-null    object 
 2   BP           100 non-null    object 
 3   Cholesterol  100 non-null    object 
 4   Na_to_K      100 non-null    float64
 5   Drug         100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [129]:
train.describe()

,Age,Na_to_K
count,100.000000,100.000000
mean,43.150000,15.634260
std,17.127788,7.096329
min,15.000000,6.683000
25%,28.750000,10.167500
50%,41.500000,13.202000
75%,57.000000,19.048500
max,74.000000,37.188000


In [130]:
test.describe()

,Age,Na_to_K
count,100.000000,100.000000
mean,45.480000,16.534710
std,15.940217,7.357364
min,15.000000,6.269000
25%,33.500000,11.152000
50%,47.000000,14.839000
75%,58.250000,20.132000
max,74.000000,38.247000


In [128]:
test.isnull().sum().sum()

np.int64(0)

In [164]:
y_train = train.pop('Drug')


In [165]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [158]:
from sklearn.preprocessing import LabelEncoder

cols = train.select_dtypes(include='object').columns

for col in cols:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])

In [149]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=0)

In [159]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred_r = rf.predict(X_val)

ValueError: Found input variables with inconsistent numbers of samples: [80, 100]

In [151]:
import lightgbm
lg = lightgbm.LGBMClassifier(random_state=0)
lg.fit(X_train, y_train)
pred_l = lg.predict(X_val)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000063 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 66
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 9
[LightGBM] [Info] Start training from score -0.916291
[LightGBM] [Info] Start training from score -1.984131
[LightGBM] [Info] Start training from score -2.436116
[LightGBM] [Info] Start training from score -3.283414
[LightGBM] [Info] Start training from score -1.086190
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [152]:
from sklearn.metrics import f1_score
f1_r = f1_score(y_val, pred_r, average='macro')
f1_l = f1_score(y_val, pred_l, average='macro')
f1_r, f1_l

(1.0, 1.0)

In [153]:
pred_r, pred_l

(array(['DrugY', 'drugX', 'DrugY', 'DrugY', 'drugX', 'DrugY', 'drugX',
        'DrugY', 'drugX', 'drugX', 'drugX', 'drugA', 'DrugY', 'DrugY',
        'drugC', 'drugA', 'drugB', 'drugX', 'DrugY', 'DrugY'], dtype=object),
 array(['DrugY', 'drugX', 'DrugY', 'DrugY', 'drugX', 'DrugY', 'drugX',
        'DrugY', 'drugX', 'drugX', 'drugX', 'drugA', 'DrugY', 'DrugY',
        'drugC', 'drugA', 'drugB', 'drugX', 'DrugY', 'DrugY'], dtype=object))

In [166]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
f1_scores = cross_val_score(rf, train, y_train, cv=3, scoring='f1_macro')
f1_scores.mean()


from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
f1_r = cross_val_score(rf, train, y_train, cv=3, scoring='f1-macro')

np.float64(0.9074643874643874)

In [167]:
from sklearn.model_selection import cross_val_score
import lightgbm
lg = lightgbm.LGBMClassifier(random_state=0)
f1_scores = cross_val_score(lg, train, y_train, cv=3, scoring='f1_macro')
f1_scores.mean()

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000039 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 58
[LightGBM] [Info] Number of data points in the train set: 66, number of used features: 8
[LightGBM] [Info] Start training from score -0.893818
[LightGBM] [Info] Start training from score -2.110213
[LightGBM] [Info] Start training from score -2.580217
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -1.054161
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB

np.float64(0.8845658367397498)

### 베이스라인

In [ ]:
# 1. 문제정의
# 평가: f1-macro
# target: Drug
# 최종파일: result.csv(컬럼 1개 pred, 1확률값)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
# train = pd.read_csv("drug_train.csv")
# test = pd.read_csv("drug_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/drug_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/drug_test.csv")

# 3. 탐색적 데이터 분석(EDA)
print("===== 데이터 정보(자료형) =====")
print(train.info())

print("\n ===== train 결측치 수 =====")
print(train.isnull().sum().sum())

print("\n ===== test 결측치 수 =====")
print(test.isnull().sum().sum())

print("\n ===== train/test 카테고리별 수 =====")
print(train[['Sex', 'BP', 'Cholesterol']].nunique())
print(test[['Sex', 'BP', 'Cholesterol']].nunique())

print("\n ===== target 빈도 =====")
print(train['Drug'].value_counts())

===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          100 non-null    int64  
 1   Sex          100 non-null    object 
 2   BP           100 non-null    object 
 3   Cholesterol  100 non-null    object 
 4   Na_to_K      100 non-null    float64
 5   Drug         100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB
None

 ===== train 결측치 수 =====
0

 ===== test 결측치 수 =====
0

 ===== train/test 카테고리별 수 =====
Sex            2
BP             3
Cholesterol    2
dtype: int64
Sex            2
BP             3
Cholesterol    2
dtype: int64

 ===== target 빈도 =====
Drug
DrugY    41
drugX    34
drugA    13
drugB     8
drugC     4
Name: count, dtype: int64


In [ ]:
# 4. 데이터 전처리
# 원핫인코딩
target = train.pop('Drug')
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import f1_score
f1 = f1_score(y_val, pred, average='macro')
print('\n f1-macro:', f1)

# 7. 예측 및 결과 파일 생성
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)


 f1-macro: 1.0


In [ ]:
# 크로스 밸리데이션(cross-validation)
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
f1_scores = cross_val_score(rf, train, target, cv=3, scoring='f1_macro')
print(f1_scores)
print(f1_scores.mean())

[1.         0.93777778 0.78461538]
0.9074643874643874


In [ ]:
# scoring에 입력 가능한 평가지표 확인 방법
from sklearn.metrics import get_scorer_names
print(get_scorer_names())

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'd2_absolute_error_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'matthews_corrcoef', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_max_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_negative_likelihood_ratio', 'neg_root_mean_squared_error', 'neg_root_mean_squared_log_error', 'normalized_mutual_info_score', 'positive_likelihood_ratio', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples'

### 성능개선

In [219]:
# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
# train = pd.read_csv("drug_train.csv")
# test = pd.read_csv("drug_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/drug_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/drug_test.csv")

# 4. 데이터 전처리
target = train.pop('Drug')

# 스케일링
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# train['Age'] = scaler.fit_transform(train[['Age']])
# test['Age'] = scaler.transform(test[['Age']])

# 원핫인코딩 (Drug 컬럼 제외)
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 레이블 인코딩 (Drug 컬럼 제외)
# target = train.pop('Drug')
# from sklearn.preprocessing import LabelEncoder
# cols = train.select_dtypes(include='object').columns
# for col in cols:
#     le = LabelEncoder()
#     train[col] = le.fit_transform(train[col])
#     test[col] = le.transform(test[col])

# 5. 크로스 밸리데이션(cross-validation)
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)
f1_scores = cross_val_score(rf, train, target, cv=3, scoring='f1_macro')
print(f1_scores.mean())

# 6. 머신러닝 학습 및 평가
rf.fit(train, target)

# 7. 예측 및 결과 파일 생성
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)

0.9074643874643874


# Section3

In [220]:
# 1. 문제정의
# 평가: f1-weighted
# target: Type
# 최종파일: result.csv(컬럼 1개 pred)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
# train = pd.read_csv("glass_train.csv")
# test = pd.read_csv("glass_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/glass_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/glass_test.csv")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      149 non-null    float64
 1   Na      149 non-null    float64
 2   Mg      149 non-null    float64
 3   Al      149 non-null    float64
 4   Si      149 non-null    float64
 5   K       149 non-null    float64
 6   Ca      149 non-null    float64
 7   Ba      149 non-null    float64
 8   Fe      149 non-null    float64
 9   Type    149 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 11.8 KB


In [221]:

y_train = train.pop('Type')

In [222]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
cols = train.select_dtypes(include=['float64', 'int64']).columns

sc = RobustScaler()
train[cols] = sc.fit_transform(train[cols])
test[cols] = sc.transform(test[cols])

train.shape, test.shape

((149, 9), (65, 9))

In [223]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score


X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=0)

In [224]:
rf = RandomForestClassifier(random_state=0, max_depth=4)
rf.fit(X_train, y_train)
pred = rf.predict(X_val)

f1 = f1_score(y_val, pred, average='weighted')
f1

0.6702003878474466

In [225]:
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result.csv',index=False)
pd.read_csv('result.csv')

,pred
0,1
1,2
2,2
3,2
4,2
...,...
60,1
61,1
62,1
63,1


### 베이스라인

In [ ]:
# 1. 문제정의
# 평가: f1-weighted
# target: Type
# 최종파일: result.csv(컬럼 1개 pred)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
# train = pd.read_csv("glass_train.csv")
# test = pd.read_csv("glass_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/glass_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/glass_test.csv")

# 3. 탐색적 데이터 분석(EDA)
print("===== 데이터 크기 =====")
print(train.shape, test.shape)

print("\n ===== train 데이터 샘플 =====")
print(train.head(1))

print("\n ===== test 데이터 샘플 =====")
print(test.head(1))

print("\n ===== 데이터 정보(자료형) =====")
print(train.info())

print("\n ===== train 결측치 수 =====")
print(train.isnull().sum().sum())

print("\n ===== test 결측치 수 =====")
print(test.isnull().sum().sum())

print("\n ===== target 빈도 =====")
print(train['Type'].value_counts())

===== 데이터 크기 =====
(149, 10) (65, 9)

 ===== train 데이터 샘플 =====
        RI     Na    Mg    Al     Si    K    Ca   Ba   Fe  Type
0  1.51829  14.46  2.24  1.62  72.38  0.0  9.26  0.0  0.0     6

 ===== test 데이터 샘플 =====
        RI     Na    Mg    Al     Si     K    Ca   Ba    Fe
0  1.51748  12.86  3.56  1.27  73.21  0.54  8.38  0.0  0.17

 ===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      149 non-null    float64
 1   Na      149 non-null    float64
 2   Mg      149 non-null    float64
 3   Al      149 non-null    float64
 4   Si      149 non-null    float64
 5   K       149 non-null    float64
 6   Ca      149 non-null    float64
 7   Ba      149 non-null    float64
 8   Fe      149 non-null    float64
 9   Type    149 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 11.8 KB
None

 ===== train 결측치 수 =====
0

 =

In [ ]:
# 4. 데이터 전처리
target = train.pop('Type')

# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import f1_score
score = f1_score(y_val, pred, average='weighted')
print('f1:', score)

# 7. 예측 및 결과 파일 생성
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)

f1: 0.6119801766860591


### 성능개선

In [ ]:
# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
# train = pd.read_csv("glass_train.csv")
# test = pd.read_csv("glass_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/glass_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch7/glass_test.csv")

# 4. 데이터 전처리
target = train.pop('Type')

# 스케일링 효과 없음

# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=5, n_estimators=200, random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import f1_score
score = f1_score(y_val, pred, average='weighted')
print('f1:', score)

# 7. 예측 및 결과 파일 생성
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)

f1: 0.6507936507936507
